<a href="https://colab.research.google.com/github/CrisTafRos/Proyecto_DSPython_Eq6/blob/main/Proyecto_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análisis de tenencias pagadas de 2008 al 2019

Año con año, el titular de un automóvil tiene que hacer el pago respectivo a su tenencia.

Estos pagos son registrados en un sistema al que solamente el área de finanzas puede acceder y únicamente compartió los montos, fechas y placas que realizaron los pagos.

Como vemos, por el mismo origen de los datos es que tenemos muchos datos y errores de captura, pero podremos llegar a depurarlos.

NOTA: Las placas utilizadas en este proyecto fueron intercambiadas con el objetivo de resguardar la información de los involucrados.

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
pagos = pd.read_csv('/content/drive/MyDrive/Data_Tenencias/tenencias_autos.csv', sep = "|", error_bad_lines = False)

b'Skipping line 52: expected 14 fields, saw 15\nSkipping line 205: expected 14 fields, saw 16\nSkipping line 250: expected 14 fields, saw 15\nSkipping line 1930: expected 14 fields, saw 17\nSkipping line 2258: expected 14 fields, saw 16\nSkipping line 4944: expected 14 fields, saw 21\nSkipping line 5252: expected 14 fields, saw 15\nSkipping line 8104: expected 14 fields, saw 15\nSkipping line 8144: expected 14 fields, saw 15\nSkipping line 9165: expected 14 fields, saw 16\nSkipping line 9166: expected 14 fields, saw 16\nSkipping line 10617: expected 14 fields, saw 15\nSkipping line 10866: expected 14 fields, saw 16\nSkipping line 12851: expected 14 fields, saw 15\nSkipping line 13263: expected 14 fields, saw 16\nSkipping line 14929: expected 14 fields, saw 22\nSkipping line 15321: expected 14 fields, saw 15\nSkipping line 15326: expected 14 fields, saw 15\nSkipping line 15388: expected 14 fields, saw 15\nSkipping line 16446: expected 14 fields, saw 15\nSkipping line 16546: expected 14 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nombres_columnas = list(pagos.columns)
placas = pd.DataFrame(pagos["Placa"])

Función para separar columnas de datos

In [ ]:
def separando_data(columna, placas):
  temp = columna.str.split(',', expand = True)
  temp = temp.iloc[0:, 0:3] # Sin esto, me regresa pura basura :(
  temp.columns = ['año_fiscal', 'monto_pagado', 'concepto']
  temp = pd.concat([placas.reset_index(drop=True), temp], axis=1)
  return temp

In [ ]:
primer_pago = separando_data(pagos[nombres_columnas[1]], placas)
segundo_pago = separando_data(pagos[nombres_columnas[2]], placas)
tercer_pago = separando_data(pagos[nombres_columnas[3]], placas)
cuarto_pago = separando_data(pagos[nombres_columnas[4]], placas)
quinto_pago = separando_data(pagos[nombres_columnas[5]], placas)
sexto_pago = separando_data(pagos[nombres_columnas[6]], placas)
septimo_pago = separando_data(pagos[nombres_columnas[7]], placas)
octavo_pago = separando_data(pagos[nombres_columnas[8]], placas)
noveno_pago = separando_data(pagos[nombres_columnas[9]], placas)
decimo_pago = separando_data(pagos[nombres_columnas[10]], placas)
decimoprimer_pago = separando_data(pagos[nombres_columnas[11]], placas)
decimosegun_pago = separando_data(pagos[nombres_columnas[12]], placas)
decimotercer_pago = separando_data(pagos[nombres_columnas[13]], placas)

In [ ]:
pagos_fin = pd.concat([primer_pago, segundo_pago, tercer_pago, cuarto_pago, quinto_pago, sexto_pago, septimo_pago, octavo_pago, noveno_pago, decimo_pago, decimoprimer_pago, decimosegun_pago, decimotercer_pago])

In [ ]:
pagos_fin = pagos_fin.sort_values('Placa')
pagos_fin

Pasando todos los errores como si fuesen NaNs

In [ ]:
df = pagos_fin # Para no perderme, desde aquí mejor asigno todo a otra variable

In [ ]:
df = df.replace('null', np.nan)
df = df.replace('', np.nan)

Quitando NaN

In [ ]:
df.isna().sum()

Placa                0
año_fiscal      203415
monto_pagado    203415
concepto        200490
dtype: int64

In [ ]:
df = df.dropna(axis = 0, how = 'any')
df.reset_index(drop=True, inplace=True)
df

,Placa,año_fiscal,monto_pagado,concepto
0,000AA,2017,1465,38
1,000AA,2016,1408,38
2,000AA,2014,1314,38
3,000AA,2013,1266,38
4,000AA,2018,1545,38
...,...,...,...,...
211904,RS508,2020,1689,38
211905,RS508,2013,1266,38
211906,RS508,2015,1366,38
211907,RS508,2019,1625,38


Clasificando por placas:


*   6 dígitos: Autos particulares. Ejemplo: A00AAA
*   5 dígitos:
  * Autos antiguos: AA000
  * Autos para personas con discapacidad: 000AA



In [ ]:
placas = pd.DataFrame(np.unique(df['Placa']))
placas.reset_index(drop=True, inplace=True)
placas

,0
0,000AA
1,000AB
2,000AD
3,000AE
4,000AF
...,...
28190,RS504
28191,RS505
28192,RS506
28193,RS507


In [ ]:
tipo_vehiculo = list()

In [ ]:
for pla in placas[0]:
  if len(pla) != 6:
    if re.findall('[0-9]', pla[0]):
      tipo_vehiculo.append('Discapacidad') #Placas con 5 caracteres y comiencen con numero es Discapacidad
    else:
      tipo_vehiculo.append('Antiguo') #Placas con 5 caracteres y comiencen con letra es Antiguo
  else:
    tipo_vehiculo.append('Particular') # Placas con 6 caracteres es Particular

In [ ]:
tipo_vehiculo = pd.DataFrame(tipo_vehiculo)

In [ ]:
placas = pd.concat([placas.reset_index(drop=True), tipo_vehiculo], axis=1)
placas.columns = ['Placa', 'tipo']
placas

,Placa,tipo
0,000AA,Discapacidad
1,000AB,Discapacidad
2,000AD,Discapacidad
3,000AE,Discapacidad
4,000AF,Discapacidad
...,...,...
28190,RS504,Antiguo
28191,RS505,Antiguo
28192,RS506,Antiguo
28193,RS507,Antiguo


In [ ]:
placas = placas.join(df.set_index('Placa'), on = 'Placa')
placas

,Placa,tipo,año_fiscal,monto_pagado,concepto
0,000AA,Discapacidad,2017,1465,38
0,000AA,Discapacidad,2016,1408,38
0,000AA,Discapacidad,2014,1314,38
0,000AA,Discapacidad,2013,1266,38
0,000AA,Discapacidad,2018,1545,38
...,...,...,...,...,...
28194,RS508,Antiguo,2020,1689,38
28194,RS508,Antiguo,2013,1266,38
28194,RS508,Antiguo,2015,1366,38
28194,RS508,Antiguo,2019,1625,38


In [ ]:
placas = placas.sort_values(['Placa', 'año_fiscal'])
placas = placas.reset_index(drop=True)
placas

,Placa,tipo,año_fiscal,monto_pagado,concepto
0,000AA,Discapacidad,2013,1266,38
1,000AA,Discapacidad,2013,1266,38
2,000AA,Discapacidad,2014,1314,38
3,000AA,Discapacidad,2015,1366,38
4,000AA,Discapacidad,2016,1408,38
...,...,...,...,...,...
211904,RS508,Antiguo,2016,1408,38
211905,RS508,Antiguo,2017,1465,38
211906,RS508,Antiguo,2018,1545,38
211907,RS508,Antiguo,2019,1625,38


In [ ]:
placas = placas.drop_duplicates()

Se solicita que se quiten los años que tienen ceros, después de la previsualización 

In [ ]:
placas = placas.replace('0', np.nan)
placas = placas.dropna(axis = 0, how = 'any')
placas

,Placa,tipo,año_fiscal,monto_pagado,concepto
0,000AA,Discapacidad,2013,1266,38
2,000AA,Discapacidad,2014,1314,38
3,000AA,Discapacidad,2015,1366,38
4,000AA,Discapacidad,2016,1408,38
5,000AA,Discapacidad,2017,1465,38
...,...,...,...,...,...
211904,RS508,Antiguo,2016,1408,38
211905,RS508,Antiguo,2017,1465,38
211906,RS508,Antiguo,2018,1545,38
211907,RS508,Antiguo,2019,1625,38


In [ ]:
placas.dtypes

Placa           object
tipo            object
año_fiscal      object
monto_pagado    object
concepto        object
dtype: object

In [ ]:
placas['monto_pagado'] = placas['monto_pagado'].astype(float)
placas['año_fiscal'] = placas['año_fiscal'].astype(int)

In [ ]:
# Hasta aquí, se exporta para poder tener un previo
placas.to_csv('/content/drive/MyDrive/Data_Tenencias/salidacero.csv', index = False)
placas

,Placa,tipo,año_fiscal,monto_pagado,concepto
0,000AA,Discapacidad,2013,1266.0,38
2,000AA,Discapacidad,2014,1314.0,38
3,000AA,Discapacidad,2015,1366.0,38
4,000AA,Discapacidad,2016,1408.0,38
5,000AA,Discapacidad,2017,1465.0,38
...,...,...,...,...,...
211904,RS508,Antiguo,2016,1408.0,38
211905,RS508,Antiguo,2017,1465.0,38
211906,RS508,Antiguo,2018,1545.0,38
211907,RS508,Antiguo,2019,1625.0,38


In [ ]:
 acumulado = placas.groupby('año_fiscal').sum()    #¿Cuánto dinero es acumulado después del pago de este impuesto por año?
 acumulado

,monto_pagado
año_fiscal,
2008,1020682.00
2009,1629165.00
2010,803832.00
2011,1957793.00
2012,457897.00
2013,32133460.00
2014,33544159.00
2015,32410505.46
2016,31165153.00


In [ ]:
diferencia = acumulado['monto_pagado'] - acumulado['monto_pagado'].shift(1)   #¿Cuál es la diferencia de ingresos obtenidos por cada periodo anual?
diferencia = pd.DataFrame(diferencia)
diferencia

,monto_pagado
año_fiscal,
2008,NaN
2009,608483.00
2010,-825333.00
2011,1153961.00
2012,-1499896.00
2013,31675563.00
2014,1410699.00
2015,-1133653.54
2016,-1245352.46


In [ ]:
placas.filter(['tipo','Placa']).groupby('tipo').count()    #¿Cuál es el tipo de placa de cada vehículo?

,Placa
tipo,
Antiguo,18352
Discapacidad,35091
Particular,131199
